In [2]:
import pandas as pd

`Rating datasets`

In [ ]:
#Import datasets of ratings
df_1= pd.read_csv(r'ratings/1.csv')
df_2= pd.read_csv(r'ratings/2.csv')
df_3= pd.read_csv(r'ratings/3.csv')
df_4= pd.read_csv(r'ratings/4.csv')
df_5= pd.read_csv(r'ratings/5.csv')
df_6= pd.read_csv(r'ratings/6.csv')
df_7= pd.read_csv(r'ratings/7.csv')
df_8= pd.read_csv(r'ratings/8.csv')

In [23]:
def get_platform(movie_id): #Addition of a new column with their correspondant platforms
    if movie_id.startswith('ns'):
        return 'netflix'
    elif movie_id.startswith('as'):
        return 'amazon'
    elif movie_id.startswith('hs'):
        return 'hulu'
    elif movie_id.startswith('ds'):
        return 'disney'
    else:
        return 'no information'

#Transformation of "ratings"
def transformation2(df1,df2,df3,df4,df5,df6,df7,df8):
    df_9= pd.concat([df1,df2,df3,df4,df5,df6,df7,df8], axis=0) #concatenate all the datasets
    df_9['timestamp'] = pd.to_datetime(df_9['timestamp'], unit='s').dt.year #change date format
    df_9= df_9.groupby(['movieId','timestamp'])['rating'].mean().round(2).reset_index() #group by movieId and timestamp
    df_9= df_9.rename(columns={'timestamp':'year'}) #rename column timestamp by year to make easier filters on the API
    df_9['platform'] = df_9['movieId'].apply(get_platform) #creation of new column "platform" that indicates where each register belongs
    df_9= df_9.set_index('movieId') #set movieId as index
    return df_9

In [ ]:
#Transformation of datasets
df_ratings= transformation2(df_1, df_2, df_3, df_4, df_5, df_6, df_7,df_8)

In [ ]:
#Export ratings datasets concatenated and grouped by year
df_ratings.to_csv('dataset_ratings.csv')

In [4]:
#Export ratings dataset as "dataset_ratings.csv" 
df_ratings= pd.read_csv(r'dataset_ratings.csv',index_col='movieId')

In [16]:
df_ratings.shape

(505909, 3)

`Plataforms datasets`

In [3]:
#Import platform datasets
df_amazon= pd.read_csv(r'datasets/amazon_prime_titles.csv')
df_disney= pd.read_csv(r'datasets/disney_plus_titles.csv')
df_hulu= pd.read_csv(r'datasets/hulu_titles.csv')
df_netflix= pd.read_csv(r'datasets/netflix_titles.csv')

In [6]:
#Define a transformation funtion with all the necessary changes
def transformation(df, word, platform):
    df['show_id']= word + df['show_id'] #Addtion of the extra word
    df['rating'].fillna('G', inplace= True) #Fill of None elements with the word "G"
    df['date_added']= pd.to_datetime(df['date_added']) #Type change of "date_added" column from Object to DateTime
    df= df.applymap(lambda s: s.lower() if type(s)== str else s) #Lower case all the Dataframe elements
    df[['duration_int', 'duration_type']]= df['duration'].str.split(' ', expand= True) #Split of "duration" columns into "duration_int" and "duration_type"
    df.insert(loc= df.columns.get_loc('duration') + 1, column='duration_int',value=df.pop('duration_int')) #Relocation of "duration_int" column
    df.insert(loc= df.columns.get_loc('duration') + 2, column='duration_type',value=df.pop('duration_type')) #Relocation of "duration_type" column
    df['duration_type']=df['duration_type'].str.replace('seasons','season') #Standarize "seasons" and "season" into "season"
    df= df.drop(columns=['duration']) #Drop of duration column
    df['platform']= platform #Column added to filter
    return df

In [7]:
#Transformation of datasets
df_amazon= transformation(df_amazon,'a','amazon')
df_disney= transformation(df_disney,'d','disney')
df_hulu= transformation(df_hulu,'h','hulu')
df_netflix= transformation(df_netflix,'n','netflix')

In [8]:
#Contatenate platform datasets
df_platform= pd.concat([df_amazon,df_disney,df_hulu,df_netflix], axis=0)

`QUERIES`

In [43]:
#1) max duration movie
def get_max_duration(year, platform, duration_type):
    max_duration = df_platform[(df_platform['platform']== platform) & (df_platform['release_year']== year) & (df_platform['duration_type']== duration_type) & (df_platform['type']=='movie')].sort_values(by= ['duration_int']).iloc[-1]
    return {'movie':max_duration['title']}

#2) Number of movies per platform with a rating > XX
def get_score_count(platform, scored, year):
    return {'platform':platform,
            'number':len(df_ratings[(df_ratings['year']== year) & (df_ratings['platform']== platform) & (df_ratings['rating']> scored)]),
            'year':year,
            'score':scored}

#3) Number of movies per platform
def get_count_platform(platform):
    return {'platform':platform, 'movies':len(df_platform[df_platform['platform']== platform])}

#4) Actor who appears the most frequently by platform and year
def get_actor(platform, year):
    by_actor = df_platform.loc[(df_platform['platform'] == platform) & (df_platform['release_year'] == year), 'cast']
    all_actors = [actor.strip() for cast_list in by_actor.str.split(',') if isinstance(cast_list, list) for actor in cast_list]
    if not all_actors: #If the column is empty
        return {'platform':platform, "year":year,'actor':'no information','times':0}
    else:
        #Most common actor and frequence
        most_common_actor = max(set(all_actors), key = all_actors.count)
        frequency = all_actors.count(most_common_actor)
        return {'platform':platform, "year":year,'actor':most_common_actor,'times':frequency}


#5) Number of content/products (everything available on streaming) that was published per country and year
def prod_per_county(type,country,year):
    return {'country':country, 'year':year, 'movies':len(df_platform[(df_platform['type']==type) & (df_platform['country']==country) & (df_platform['release_year']== year)])}

#6) Total number of content/products (everything available on streaming, series, documentaries, movies, etc.)
#according to the given audience rating (for which audience the movie was classified).
def get_contents(rating):
    return {'rating':rating,'content':len(df_platform[(df_platform['rating']==rating)])}

In [44]:
print(get_max_duration(2015,'disney','min')) #1
print(get_score_count('hulu', 2.5, 2002)) #2
print(get_count_platform('disney')) #3
print(get_actor('hulu',2019)) #4
print(prod_per_county('movie','india',2018)) #5
print(get_contents('g')) #6

{'movie': 'inside out'}
{'platform': 'hulu', 'number': 3070, 'year': 2002, 'score': 2.5}
{'platform': 'disney', 'movies': 1450}
{'platform': 'hulu', 'year': 2019, 'actor': 'no information', 'times': 0}
{'country': 'india', 'year': 2018, 'movies': 112}
{'rating': 'g', 'content': 1269}
